### Credit Card Customers

Importing all necessary modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import PowerTransformer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

import category_encoders as ce

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import AdaBoostClassifier

from scipy import stats
import statsmodels.stats.power as power

from imblearn.pipeline import Pipeline as Pipe
from itertools import combinations
from imblearn.combine import SMOTEENN

from catboost import Pool, CatBoostClassifier
from lightgbm import LGBMClassifier
import optuna
from optuna.samplers import TPESampler

import category_encoders as ce
!pip install -q BorutaShap
from BorutaShap import BorutaShap
import shap

np.random.seed(123)
clear_output()

In [2]:
# defining the path to the dataset and target name 
data_root = '../input/credit-card-customers/BankChurners.csv'
TARGET_NAME = 'Attrition_Flag'

In [19]:
# load dataset in dataframe
df = pd.read_csv(data_root)

Next we define helper functions:
* show_proba_calibration_plots: displays probability threshold, that helps to increase recall, precision or F-score
* report: displays the estimators results - metrics
* reduce_memory: transforms the input pandas dataframe dtypes into lower memory size if possible
* plot_ROC: displays Area under curve plots to visualize roc_auc score
* vote: helps to apply majority voting if we use multiple estimators
* cross_validation: displays cross_validation results and mean score
* statistic_output, effect_size, categorical_stats: helper functions to make statistical tests as for numerical rather than categorical features to see if they have any statistical impact on target variable

In [33]:
def show_proba_calibration_plots(y_predicted_probs, y_true_labels):
    preds_with_true_labels = np.array(list(zip(y_predicted_probs, y_true_labels)))

    thresholds = []
    precisions = []
    recalls = []
    f1_scores = []

    for threshold in np.linspace(0.1, 0.9, 9):
        thresholds.append(threshold)
        precisions.append(precision_score(y_true_labels, list(map(int, y_predicted_probs > threshold))))
        recalls.append(recall_score(y_true_labels, list(map(int, y_predicted_probs > threshold))))
        f1_scores.append(f1_score(y_true_labels, list(map(int, y_predicted_probs > threshold))))

    scores_table = pd.DataFrame({'f1':f1_scores,
                                 'precision':precisions,
                                 'recall':recalls,
                                 'probability':thresholds}).sort_values('f1', ascending=False).round(3)
  
    figure = plt.figure(figsize = (15, 5))

    plt1 = figure.add_subplot(121)
    plt1.plot(thresholds, precisions, label='Precision', linewidth=4)
    plt1.plot(thresholds, recalls, label='Recall', linewidth=4)
    plt1.plot(thresholds, f1_scores, label='F1', linewidth=4)
    plt1.set_ylabel('Scores')
    plt1.set_xlabel('Probability threshold')
    plt1.set_title('Probabilities threshold calibration')
    plt1.legend(bbox_to_anchor=(0.25, 0.25))   
    plt1.table(cellText = scores_table.values,
               colLabels = scores_table.columns, 
               colLoc = 'center', cellLoc = 'center', loc = 'bottom', bbox = [0, -1.3, 1, 1])
    plt2 = figure.add_subplot(122)
    plt2.hist(preds_with_true_labels[preds_with_true_labels[:, 1] == 0][:, 0], 
              label='Another class', color='royalblue', alpha=1)
    plt2.hist(preds_with_true_labels[preds_with_true_labels[:, 1] == 1][:, 0], 
              label='Main class', color='darkcyan', alpha=0.8)
    plt2.set_ylabel('Number of examples')
    plt2.set_xlabel('Probabilities')
    plt2.set_title('Probability histogram')
    plt2.legend(bbox_to_anchor=(1, 1))

    plt.show()

def report(y_train, y_train_pred, y_test, y_test_pred, y_train_proba=None, y_test_proba=None):
    print('Train\n', classification_report(y_train, y_train_pred, digits=3))
    print('Test\n', classification_report(y_test, y_test_pred, digits=3))
    if y_train_proba is not None and y_test_proba is not None:
        roc_train, roc_test = roc_auc_score(y_train, y_train_proba), roc_auc_score(y_test, y_test_proba)
        print(f'Train ROC_AUC: {roc_train:.3f}, Test ROC_AUC: {roc_test:.3f}')
    print('Confusion Matrix', '\n', pd.crosstab(y_test, y_test_pred))

def reduce_memory(df, verbose=0):
    if verbose != 0:
        start_mem = df.memory_usage().sum() / 1024 ** 2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object and str(col_type)[:4] != 'uint' and str(col_type) != 'category':
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif str(col_type)[:4] != 'uint':
            df[col] = df[col].astype('category')
    if verbose != 0:
        end_mem = df.memory_usage().sum() / 1024 ** 2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

def roc_plot(y_true, probs):
    fpr, tpr, threshold = roc_curve(y_true, probs)
    roc_auc = auc(fpr, tpr)
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

def vote(predictions: list, weights: list):
    predictions = np.asarray(predictions).T
    maj_vote = np.apply_along_axis(lambda x: np.argmax(np.bincount(x, weights=weights)), axis=1, arr=predictions)
    return maj_vote

def cross_validation(clf, X, y, scoring='f1'):
    scores = cross_val_score(estimator=clf, X=X, y=y, cv=10, scoring=scoring, n_jobs=-1)
    print(f'Меры правильности перекрекстной оценки: {scores}')
    print(f'Точность перекретсной оценки: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')
    return scores

def effect_size(factor_a, factor_b, cohen=True, desired_power=0.8, alpha=0.05):
    n1, n2 = len(factor_a), len(factor_b)
    s1, s2 = factor_a.std(ddof=1), factor_b.std(ddof=1)
    df = (s1 ** 2 / n1 + s2 ** 2 / n2) ** 2 / \
          ((s1 ** 2 / n1) ** 2 / (n1 - 1) + (s2 ** 2 / n2) ** 2 / (n2 - 1))
    if cohen:
        sigma_pooled = np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
        return np.abs(factor_a.mean() - factor_b.mean()) / sigma_pooled, df
    else:
        return power.tt_ind_solve_power(effect_size=None, nobs1=len(factor_a), alpha=alpha, power=desired_power,
                                        ratio=len(factor_b)/len(factor_a), alternative='two-sided'), df
    
def statistic_output(*columns, df=df, cat=False, target=TARGET_NAME, alpha=0.05, sample_size=0):
    data = df.copy()
    data.drop_duplicates(inplace=True)
    if sample_size == 0:
        sample_size = int(0.05 * len(data))
    if not cat:
        columns = data.drop(target, axis=1).select_dtypes(exclude=['category', 'object']).columns
        for column in columns:
            df_sampled = data[[column, target]].sample(sample_size, random_state=1)
            factor_a = df_sampled.loc[df_sampled[target] == 0][column]   
            factor_b = df_sampled.loc[df_sampled[target] == 1][column]
            var_a, var_b = factor_a.var(), factor_b.var()   
            _, pvalue = stats.shapiro(df_sampled[column])
            if pvalue >= alpha:
                _, pvalue = stats.ttest_ind(factor_a, factor_b, equal_var=False)
                test = power.TTestIndPower()
                eff_size, deg_free = effect_size(factor_a, factor_b, cohen=False)
                pow = test.power(effect_size=eff_size, nobs1=len(factor_a), alpha=alpha, df=deg_free, 
                                 ratio=len(factor_b)/len(factor_a), alternative='two-sided')
            else:
                _, pvalue = stats.mannwhitneyu(factor_a, factor_b)
                pow, eff_size = None, None
            if pvalue < alpha:
                result = f'with effect_size = {eff_size:.4f} and ttest power {pow*100:.2f}%' if pow is not None else ''
                print(f'Factor "{column}" has statistical impact on target (var_a: {var_a:.2f}, var_b: {var_b:.2f}). {result}')
            else:
                print(f'Factor "{column}" does not affect target.')
    else:
        for column in columns:
            print(column)
            categories = data[column].unique().tolist()
            for pair in combinations(categories, r=2):
                a, b = pair
                if a != b:
                    data_ = data.loc[data[column].isin(pair), ['CLIENTNUM', column, target]].sample(sample_size, random_state=1)
                    table = data_.pivot_table(values='CLIENTNUM', index=column, columns=target, aggfunc='count')
                    try:
                        _, pvalue, _, _ = stats.chi2_contingency(table, correction=False)
                    except ValueError:
                        continue
                    if pvalue >= alpha:
                        print(f'Categories {a} and {b} can be united. P-value: {pvalue:.6f}')
                    else:
                        print(f'Categories {a} and {b} have different frequencies with target.')
                        
def categorical_stats(df=df, target=TARGET_NAME, alpha=0.05, sample_size=500):
    data = df.copy().sample(sample_size)
    columns_to_analize = data.select_dtypes(include=['category', 'object']).columns
    weak_list = []
    for factor in columns_to_analize:
        if factor == target:
            continue
        print(f'{factor}')
        table = pd.crosstab(data[factor], data[TARGET_NAME])
        p_value = stats.chi2_contingency(table, correction=False)[1]
        if p_value < alpha:
            print(f'Feature {factor} has statistical impact on target. P-value: {p_value:.6f}')
        else:
            weak_list.append(factor)
    if len(weak_list) > 0:
        print(f'Statistically weak categorical features: ', *weak_list)

### Some EDA and data visualisations

In [5]:
df.sample(8).transpose()

In [26]:
# remove two last columns as the author suggests
df = df.iloc[:, :-2]
print(f'Total columns {len(df.columns)}')

# check for duplicates
print(f'Number of duplicates: {df.duplicated().astype(int).sum()}')
df.drop_duplicates(inplace=True)

In [8]:
# common characteristics for numerical features: 
# it does not seem, that there any extreme values in data
# some of the columns may follow normal distribution as their means and medians are close
df.describe()

In [9]:
# common characteristics for categorical features
# there are not high cardinality in here, seems it would be fine to use one-hot encoding
df.describe(include=['object'])

In [29]:
# overall statistics
# good news: there are no missing values in dataset (missingno visualisation is not needed)
df.info()

In [11]:
# we can observe class imbalance in data
# next we will try three ways to handle it after predicting on raw data with base model
# 1) SMOTEENN - that generates new sinthetical samples and discards samples that are close
# to the decision boundary
# 2) Use in-built class_weight attributes in models (as alt. sklearn provides a method:
# sklearn.utils.class_weight.compute_class_weight) to penalize the loss function
# 3) Hybrid of sampling (SMOTE) and class_weight methods
df[TARGET_NAME].value_counts(normalize=True)

In [20]:
# let's reduce memory usage to speed up and optimize our calculations
df = reduce_memory(df, verbose=123)

#### Some useful data visualizations

In [13]:
numerical_cols = df.select_dtypes(include=[np.int8, np.int16, np.float32]).columns.tolist()

plt.figure(figsize=(22, 20))
for idx, column in enumerate(numerical_cols):
    plt.subplot(4, 4, idx + 1)
    dist = 'Normal Distribution' if stats.shapiro(df[column].sample(1000))[1] > 0.05 else 'Not normal distribution'
    plt.title(f'{column}: {dist}')
    sns.histplot(data=df, x=column, hue=TARGET_NAME, bins=60, kde=True)
plt.subplots_adjust(hspace=0.4, wspace=0.4)
plt.show()
# Age is close to normal distribution
# We can clearly see that numerical data has different distribution via target
# also there are strange tail in Credit_Limit and Total_Revolvong_Bal
# we will inspect them with IQR boxplots
# months_on_book have large amount of median values
# also we have to admit that there are some count features, which sounds good for 
# boosting models

In [14]:
categorical_cols = df.drop(TARGET_NAME, axis=1).select_dtypes(include=['category']).columns

plt.figure(figsize=(15, 10))
for idx, column in enumerate(categorical_cols, 1):
    plt.subplot(2, 3, idx)
    plt.title(f'{column}')
    sns.countplot(x=column, hue=TARGET_NAME, data=df)
plt.subplots_adjust(hspace=0.4, wspace=0.4)
plt.show()
# for the first look there are no any significant insites
# for card_category it is better to apply label smoothing if not using one-hot encoding

In [15]:
# we do not see any extreme outliers here
# Inspect: every column, except Customer_Age, Total_Relationship_Count,Total_Revolving_Bal
# we will try next after predicting on raw data:
# 1) IQR adjusting
# 2) PowerTransformer to make the data more normal and less skewed
# 3) IterativeImputer to impute possible outliers, detected by IQR
# NOTE: also good techniques for outlier detection are IsolationForest, DBSCAN, 
# dimensionality reduction methods

plt.figure(figsize=(18,20))
for idx, column in enumerate(numerical_cols, 1):
    plt.subplot(4, 4, idx)
    sns.boxplot(y=df[column], x=df[TARGET_NAME], data=df)
    plt.title(f'{column}')
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.show()

In [16]:
# let's see what linear correlations between features do we have
# we have significant correlation between Customer_Age and Months_on_book,
# Total_Trans_Amt and Total_Trans_Ct
# meddle high correlations between: Total_Revolving_Bal and Avg_Utilization_Ratio
# Avg_Open_To_Buy and Avg_Utilization_Ratio
# looks like the author created a few variables using groupby and count, averaging 
# methods
# As the correlations are high it might be useful to drop one of the dependent feature
# but we would rather use feature-selection methods to decide whether to drop some, 
# as otherwise we can loose meaningful information
plt.figure(figsize = (14,12))
corr_matrix = df.corr()
corr_matrix = np.round(corr_matrix, 2)
corr_matrix[np.abs(corr_matrix) < 0.3] = 0
sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='coolwarm')
plt.title('Correlation matrix')
plt.show()

#### Let's compute some statistical tests to find out the significance of given variables

In [21]:
df[TARGET_NAME] = df[TARGET_NAME].map({'Existing Customer': 0, 'Attrited Customer': 1})
df[TARGET_NAME] = df[TARGET_NAME].astype(np.uint8)

In [34]:
# first we will check the histplot results, assuming all numerical features are useful
# we got some interesting insites! For a given significance level (0.05) and sample_size
# (5%) we can conclude that Customer_Age, Dependent_count, Months_on_book, Avg_Open_To_Buy
# have no statisticaly meaningful impact on target
# again: we will note this fact in head and later apply feature-selection to decide
statistic_output(*numerical_cols)

In [23]:
# we use here only gender, as it is not enough category samples for such type of test
# But again we see that Gender has no impact on target as there are two equally 
# distributed categories
statistic_output('Gender', cat=True, sample_size=500)

In [24]:
# with statistic output we decide whether it is possible to union two taken categories
# here we determin if there are statistical dependency between two 
# variables (categorical feature and target)
# we got ALL statistically weak features. In the end, we cannot drop them all
# we have to make feature-selection
categorical_stats()

#### Complete Raw dataset and define base model

In [35]:
# set id as index as we won't use it in predictions, but also we want to 
# identify every customer later
df.set_index('CLIENTNUM', drop=True, inplace=True)
df.head(1)

In [36]:
# just mapping binary caterories and changing the correct dtype
df['Gender'] = df['Gender'].map({'M': 0, 'F': 1})
df['Gender'] = df['Gender'].astype(np.uint8)

In [37]:
# simply one-hot encode categorical features
df = pd.get_dummies(df, prefix=['col1', 'col2', 'col3', 'col4'])
df.shape

In [38]:
# getting all model parameters as default
base = LGBMClassifier(verbose=-1)

#### Search of the best split

In [39]:
X, y = df.drop(TARGET_NAME, axis=1), df[TARGET_NAME]

train_sizes, train_scores, test_scores = learning_curve(estimator=base, X=X, y=y, 
                                                        train_sizes=np.linspace(0.1, 1.0, 10), cv=10, scoring='roc_auc', n_jobs=-1)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
plt.figure(figsize=(10, 8))
plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='правильность при обучении')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', linestyle='--', marker='s', markersize=5, 
         label='правильность при проверке')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.axvline(X.shape[0]*0.7, color='orange', linestyle='-.', label='test_size=0.3')
plt.grid()
plt.xlabel('Количество обучающих образцов')
plt.ylabel('Правильность')
plt.legend(loc='best')
plt.show()
# setting test_size=0.3 looks fine, but in case of validation data we will set
# test_size=0.25, valid_size=0.15

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, stratify=y_train, random_state=123)
print(f'Train size: {X_train.shape[0]}, Validation size: {X_valid.shape[0]}, Test size: {X_test.shape[0]}')

#### Dealing with outliers

In [41]:
# raw predictions
base.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, early_stopping_rounds=5, eval_metric='roc_auc')
base_raw_train = base.predict(X_train)
base_raw_test = base.predict(X_test)
base_train_proba = base.predict_proba(X_train)[:,1]
base_test_proba = base.predict_proba(X_test)[:,1]

report(y_train, base_raw_train, y_test, base_raw_test, base_train_proba, base_test_proba)

In [42]:
columns_with_outliers = ['Dependent_count','Months_on_book', 'Months_Inactive_12_mon',
                         'Contacts_Count_12_mon', 'Credit_Limit', 'Avg_Open_To_Buy',
                         'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct',
                         'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']

In [43]:
# IQR adjusting
for column in columns_with_outliers:
    q1 = np.quantile(X_train[column], 0.25)
    q3 = np.quantile(X_train[column], 0.75)
    iq_low = q1 - 1.5 * (q3 - q1)
    iq_high = q3 + 1.5 * (q3 - q1)
    X_train.loc[X_train[column] > iq_high, column] = iq_high
    X_train.loc[X_train[column] < iq_low, column] = iq_low
    
    X_valid.loc[X_valid[column] > iq_high, column] = iq_high
    X_valid.loc[X_valid[column] < iq_low, column] = iq_low
    
    X_test.loc[X_test[column] > iq_high, column] = iq_high
    X_test.loc[X_test[column] < iq_low, column] = iq_low

In [44]:
base.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, early_stopping_rounds=5, eval_metric='roc_auc')
base_iqr_train = base.predict(X_train)
base_iqr_test = base.predict(X_test)
base_train_proba = base.predict_proba(X_train)[:,1]
base_test_proba = base.predict_proba(X_test)[:,1]

report(y_train, base_iqr_train, y_test, base_iqr_test, base_train_proba, base_test_proba)
# actually we cannot see any meaningful difference. Thus FN is higher than in raw-case
# We assume that it is better to leave less FN

In [46]:
# Transformation to normal distribution
column_transformer = ColumnTransformer(transformers=[
    ('power', PowerTransformer(), columns_with_outliers)
], remainder='passthrough')

X_train = column_transformer.fit_transform(X_train, y_train)
X_valid = column_transformer.transform(X_valid)
X_test = column_transformer.transform(X_test)

In [47]:
base.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, early_stopping_rounds=5, eval_metric='roc_auc')
base_pow_train = base.predict(X_train)
base_pow_test = base.predict(X_test)
base_train_proba = base.predict_proba(X_train)[:,1]
base_test_proba = base.predict_proba(X_test)[:,1]

report(y_train, base_pow_train, y_test, base_pow_test, base_train_proba, base_test_proba)
# as excpected normal-transformation method does not fit this particular situation

In [49]:
# Imputation
for column in columns_with_outliers:
    q1 = np.quantile(X_train[column], 0.25)
    q3 = np.quantile(X_train[column], 0.75)
    iq_low = q1 - 1.5 * (q3 - q1)
    iq_high = q3 + 1.5 * (q3 - q1)
    X_train.loc[X_train[column] > iq_high, column] = None
    X_train.loc[X_train[column] < iq_low, column] = None
    
    X_valid.loc[X_valid[column] > iq_high, column] = None
    X_valid.loc[X_valid[column] < iq_low, column] = None
    
    X_test.loc[X_test[column] > iq_high, column] = None
    X_test.loc[X_test[column] < iq_low, column] = None

In [50]:
from sklearn.linear_model import LinearRegression

imputer = IterativeImputer(estimator=LinearRegression(), random_state=123)
X_train = imputer.fit_transform(X_train)
X_valid = imputer.transform(X_valid)
X_test = imputer.transform(X_test)

In [51]:
base.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, early_stopping_rounds=5, eval_metric='roc_auc')
base_imp_train = base.predict(X_train)
base_imp_test = base.predict(X_test)
base_train_proba = base.predict_proba(X_train)[:,1]
base_test_proba = base.predict_proba(X_test)[:,1]

report(y_train, base_imp_train, y_test, base_imp_test, base_train_proba, base_test_proba)

##### Also we may just drop outliers, but as if they are not so extreme, we may loose important information

### Feature Engeneering
#### We are using no more features, as there are a lot of counts and some kind of synthetic features
Grouping by categories with averaging functions and counting is good practice for boosting models.
Linear transform is good for Linear models: LogisticRegression, LinearSVM, etc.

#### Feature Selection

Feature Selection will have two different methods after predicting on raw data:
1) A class that selects the best subset of features, according to the given score, 
straightforward method, only computationaly reasonable for not large datasets

2) BorutaShap object, that will select features to remove: uses statistical tests and
shap values with given model (estimator)
we will compare the EDA results with this approach

In [52]:
class BestSet(BaseEstimator, TransformerMixin):
    def __init__(self, k_features=12, scoring=f1_score, test_size=0.2):
        self.scoring = scoring
        self.k_features = k_features
        self.test_size = test_size

    def fit(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y, 
                                                            test_size=self.test_size, 
                                                            stratify=y,
                                                            random_state=1)
        dim = X_train.shape[1]
        self.indices_ = tuple(range(dim))
        self.subsets_ = [self.indices_]
        score = self._calc_score(X_train, y_train, X_test, y_test, self.indices_)
        self.scores_ = [score]

        while dim > self.k_features:
            scores, subsets = [], []
            for p in combinations(self.indices_, r=dim-1):
                score = self._calc_score(X_train, y_train, X_test, y_test, p)
                scores.append(score)
                subsets.append(p)
            best = np.argmax(scores)
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1
            self.scores_.append(scores[best])
        self.k_score_ = self.scores_[-1]
        return self
    def transform(self, X):
        best_indices = self.subsets_[np.argmax(self.scores_)]
        return X[:, best_indices]

    def _calc_score(self, X_train, y_train, X_test, y_test, indices):
        model = LGBMClassifier(boosting_type='goss',
                               is_unbalance=True,
                               max_depth=3,
                               verbose=-1,
                               random_state=1)
        model.fit(X_train[:, indices], y_train, verbose=False)
        y_pred = model.predict(X_test[:, indices])
        score = self.scoring(y_test, y_pred)
        return score

In [54]:
pipe = Pipeline(steps=[
    ('selector', BestSet(k_features=30, scoring=roc_auc_score)),
    ('base_model', LGBMClassifier(verbose=-1))
])
fit_params = {'base_model__verbose': False}

pipe.fit(X_train.values, y_train, **fit_params)
pipe_train = pipe.predict(X_train.values)
pipe_test = pipe.predict(X_test.values)
pipe_train_proba = pipe.predict_proba(X_train.values)[:,1]
pipe_test_proba = pipe.predict_proba(X_test.values)[:,1]

report(y_train, pipe_train, y_test, pipe_test, pipe_train_proba, pipe_test_proba)
# we got a tiny improve in FN

In [55]:
indices = pipe[0].subsets_[np.argmax(pipe[0].scores_)]
print(f'Remain columns: {len(indices)}')
# dropped 4 features
X.iloc[0, list(indices)]
# it is curious as Gender feature was not dropped

In [57]:
# borutaShap
selector = BorutaShap(model=LGBMClassifier(verbose=-1), importance_measure='shap', classification=True)
selector.fit(X_train, y_train, n_trials=50, sample=False, verbose=False)

In [58]:
selector.features_to_remove

In [62]:
X_train.drop(selector.features_to_remove.tolist(), axis=1, inplace=True)
X_valid.drop(selector.features_to_remove.tolist(), axis=1, inplace=True)
X_test.drop(selector.features_to_remove.tolist(), axis=1, inplace=True)

In [63]:
base.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, early_stopping_rounds=5, eval_metric='roc_auc')
base_bor_train = base.predict(X_train)
base_bor_test = base.predict(X_test)
base_train_proba = base.predict_proba(X_train)[:,1]
base_test_proba = base.predict_proba(X_test)[:,1]

report(y_train, base_bor_train, y_test, base_bor_test, base_train_proba, base_test_proba)
# after dropping we got almost the same confusion matrix but model overfits more
# if it was large dataset it would be the best choice in computational order and in order
# to reduce the job for scrapping the data.
# So we will use BestSet with lgbm and raw data with CatBoostClassifier

#### SMOTE and class weighting

In [65]:
# class weights balancing
pipe = Pipeline(steps=[
    ('selector', BestSet(k_features=30, scoring=roc_auc_score)),
    ('base_model', LGBMClassifier(is_unbalance=True, verbose=-1))
])
fit_params = {'base_model__verbose': False}

pipe.fit(X_train.values, y_train, **fit_params)
pipe_train = pipe.predict(X_train.values)
pipe_test = pipe.predict(X_test.values)
pipe_train_proba = pipe.predict_proba(X_train.values)[:,1]
pipe_test_proba = pipe.predict_proba(X_test.values)[:,1]

report(y_train, pipe_train, y_test, pipe_test, pipe_train_proba, pipe_test_proba)

In [66]:
# single SMOTE
pipe = Pipe(steps=[
    ('selector', BestSet(k_features=30, scoring=roc_auc_score)),
    ('SMOTE', SMOTEENN(sampling_strategy=1., random_state=123)),
    ('base_model', LGBMClassifier(verbose=-1))
])
fit_params = {'base_model__verbose': False}

pipe.fit(X_train.values, y_train, **fit_params)
pipe_train = pipe.predict(X_train.values)
pipe_test = pipe.predict(X_test.values)
pipe_train_proba = pipe.predict_proba(X_train.values)[:,1]
pipe_test_proba = pipe.predict_proba(X_test.values)[:,1]

report(y_train, pipe_train, y_test, pipe_test, pipe_train_proba, pipe_test_proba)

In [67]:
# SMOTE and class_weights
pipe = Pipe(steps=[
    ('selector', BestSet(k_features=30, scoring=roc_auc_score)),
    ('SMOTE', SMOTEENN(sampling_strategy=0.7, random_state=123)),
    ('base_model', LGBMClassifier(is_unbalance=True, verbose=-1))
])
fit_params = {'base_model__verbose': False}

pipe.fit(X_train.values, y_train, **fit_params)
pipe_train = pipe.predict(X_train.values)
pipe_test = pipe.predict(X_test.values)
pipe_train_proba = pipe.predict_proba(X_train.values)[:,1]
pipe_test_proba = pipe.predict_proba(X_test.values)[:,1]

report(y_train, pipe_train, y_test, pipe_test, pipe_train_proba, pipe_test_proba)

#### We will use only class_weights

### LGBM Hyperparameter tuning

In [68]:
# in order not to select every trial
X_train = pipe[0].fit_transform(X_train.values, y_train)
X_valid = pipe[0].transform(X_valid.values)
X_test = pipe[0].transform(X_test.values)

In [ ]:
model_params = {
                'objective': 'binary', # cross_entropy
                'n_estimators': 800,
                'n_jobs': -1,
                'is_unbalance': True,
                'random_state': 123
}
fit_params = {'early_stopping_rounds': 10,  
              'eval_set': [(X_valid, y_valid)],  
              'eval_metric': 'auc',
              'verbose': False
}

In [ ]:
def objective(trial):
    param_trials = {
                    'max_depth': trial.suggest_int('max_depth', 3, 9),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
                    'reg_alpha': trial.suggest_float('reg_alpha', 1., 5.),
                    'reg_lambda': trial.suggest_float('reg_lambda', 1., 5.),
                    'num_leaves': trial.suggest_int('num_leaves', 20, 265),
                    'subsample': trial.suggest_float('subsample', 0.3, 1.),
                    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.),
                    'max_bin': trial.suggest_int('max_bin', 60, 260),
                    'min_child_samples': trial.suggest_int('min_child_samples', 120, 260),
                    'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 0.1),
                    'boosting_type': trial.suggest_categorical('boosting_type', ['goss', 'gbdt']),
                    }
    param_trials.update(model_params)
    opt_model = LGBMClassifier(**param_trials)
    opt_model.fit(X_train, y_train, **fit_params)
    
    y_pred = opt_model.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    
    return score

In [ ]:
optuna.logging.set_verbosity(optuna.logging.FATAL)
study = optuna.create_study(sampler=TPESampler(seed=123), direction="maximize")
study.optimize(objective, n_trials=300, timeout=6000)

print(f'Number of completed trials: {len(study.trials)}')
print('Best trial')
trial = study.best_trial
print(f'Best score: {trial.value}')
print('Best params')
for key, value in trial.params.items():
    print(f'{key}: {value}')

### LGBM stratified predcitions

In [69]:
model_params = {
                'objective': 'binary',
                'n_estimators': 800,
                'n_jobs': -1,
                'is_unbalance': True,
                'random_state': 123,
                'max_depth': 8,
                'learning_rate': 0.06334858903177704, # 0.16334858903177704
                'reg_alpha': 1.7581994842893593,
                'reg_lambda': 4.962605889056071,
                'num_leaves': 144,
                'subsample': 0.35154539457722855,
                'colsample_bytree': 0.5436963612580455, # 0.6436963612580455
                'max_bin': 219,
                'min_child_samples': 124,
                'min_child_weight': 0.02483278328435734,
                'boosting_type': 'gbdt',
}
fit_params = {'early_stopping_rounds': 10,  
              'eval_set': [(X_valid, y_valid)],  
              'eval_metric': 'auc',
              'verbose': False
}

In [70]:
# save index
indices = pipe[0].subsets_[np.argmax(pipe[0].scores_)]
X_index = X.index
X_ = X.values[:, indices]
X_fold, X_test, y_fold, y_test = train_test_split(X_, y, test_size=0.2, shuffle=True, stratify=y, random_state=123)
print(f'Train size: {X_fold.shape[0]}, Test size: {X_test.shape[0]}')

In [71]:
# get result probabilities on 3 folds
test_results, train_roc, test_pred, train_f1, train_acc = [], [], [], [], []
skf = StratifiedKFold(n_splits=3)

for train_index, valid_index in skf.split(X_fold, y_fold):
    X_train, X_valid = X_fold[train_index, :], X_fold[valid_index, :]
    y_train, y_valid = y_fold.values[train_index], y_fold.values[valid_index]

    lgbm = LGBMClassifier(**model_params)
    fit_params = {'early_stopping_rounds': 10,  
              'eval_set': [(X_valid, y_valid)],  
              'eval_metric': 'auc',
              'verbose': False
              }
    lgbm.fit(X_train, y_train, **fit_params)
    test_labels = lgbm.predict(X_test)
    test_pred.append(test_labels)
    
    train_proba = lgbm.predict_proba(X_train)[:,1]
    train_roc.append(roc_auc_score(y_train, train_proba))
    
    train_labels = lgbm.predict(X_train)
    train_f1.append(f1_score(y_train, train_labels))
    train_acc.append(accuracy_score(y_train, train_labels))
    
    pred_test = lgbm.predict_proba(X_test)[:,1]
    test_results.append(pred_test)

final_test = np.array(test_results).mean(axis=0)
test_pred = vote(test_pred, weights=[1.,1.,1.])

In [72]:
print(f'train mean accuracy: {np.array(train_acc).mean():.4f}, test accuracy: {accuracy_score(y_test, test_pred):.4f}')
print(f'train mean f1: {np.array(train_f1).mean():.4f}, test accuracy: {f1_score(y_test, test_pred):.4f}')
print(f'train mean auc: {np.array(train_roc).mean():.4f}, test auc: {roc_auc_score(y_test, final_test):.4f}')
# the model overfits a bit, but not critically

In [73]:
# let's see if calibration is needed
# to get higher f1, recall and precision we have to 
show_proba_calibration_plots(final_test, y_test)

### CatBoost Model
Here we take all the same preprocessing steps except one-hot encoding

In [88]:
# let's repeat all necessary steps
df = pd.read_csv(data_root) # read
df = df.iloc[:, :-2] # remove redundant columns
df.drop_duplicates(inplace=True) # drop duplicates
df = reduce_memory(df, verbose=0) # reduce memory
df[TARGET_NAME] = df[TARGET_NAME].map({'Existing Customer': 0, 'Attrited Customer': 1}) # map target
df[TARGET_NAME] = df[TARGET_NAME].astype(np.uint8)
df.set_index('CLIENTNUM', drop=True, inplace=True) # set client id as index
# split
X, y = df.drop(TARGET_NAME, axis=1), df[TARGET_NAME]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, stratify=y_train, random_state=123)
print(f'Train size: {X_train.shape[0]}, Validation size: {X_valid.shape[0]}, Test size: {X_test.shape[0]}')

In [75]:
# categorical columns; we are using Pool as a good practice, it is not necessary here as the dataset is small
cat_cols = X_train.select_dtypes(include=['category']).columns.tolist()

train_pool = Pool(X_train, y_train, cat_features=cat_cols)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_cols)

In [ ]:
params_model = {
    'eval_metric': 'AUC', 
    'iterations': 1000,
    'auto_class_weights': 'Balanced',
    'silent': True,
    'one_hot_max_size': 15,
    'early_stopping_rounds': 10,
    'grow_policy': 'SymmetricTree',
    'allow_writing_files': False,
    'use_best_model': True,
    'random_seed': 123,
    }

In [ ]:
def objective(trial):
    param_trials = {
                    'depth': trial.suggest_int('depth', 3, 9),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
                    'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1., 5.),
                    'bagging_temperature': trial.suggest_float('bagging_temperature', 1., 3.),
                    'subsample': trial.suggest_float('subsample', 0.3, 1.),
                    'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.3, 1.),
                    'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
                    }
    param_trials.update(params_model)
    opt_model = CatBoostClassifier(**param_trials)
    opt_model.fit(train_pool, eval_set=valid_pool)
    
    y_pred = opt_model.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    
    return score

In [ ]:
study = optuna.create_study(sampler=TPESampler(seed=123), direction="maximize")
study.optimize(objective, n_trials=300, timeout=6000)

print(f'Number of completed trials: {len(study.trials)}')
print('Best trial')
trial = study.best_trial
print(f'Best score: {trial.value}')
print('Best params')
for key, value in trial.params.items():
    print(f'{key}: {value}')

In [76]:
params_model = {
                'eval_metric': 'AUC', 
                'iterations': 1000,
                'auto_class_weights': 'Balanced',
                'silent': True,
                'one_hot_max_size': 15,
                'early_stopping_rounds': 5,
                'grow_policy': 'SymmetricTree',
                'allow_writing_files': False,
                'use_best_model': True,
                'random_seed': 123,
                'depth': 4,
                'learning_rate': 0.3741239438544723, 
                'l2_leaf_reg': 3.1351883224555164,
                'bagging_temperature': 2.070491556934441, 
                'subsample': 0.4848286374085917, 
                'colsample_bylevel': 0.4501924728694299, 
                'boosting_type': 'Plain', # 'Ordered'
                }

In [77]:
X_fold, X_test, y_fold, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=123)

In [78]:
# get result probabilities on 3 folds
test_results, train_roc, test_pred, train_f1, train_acc = [], [], [], [], []
skf = StratifiedKFold(n_splits=3)

for train_index, valid_index in skf.split(X_fold, y_fold):
    X_train, X_valid = X_fold.iloc[train_index, :], X_fold.iloc[valid_index, :]
    y_train, y_valid = y_fold.iloc[train_index], y_fold.iloc[valid_index]

    cat = CatBoostClassifier(cat_features=cat_cols, **params_model)

    cat.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
    test_labels = cat.predict(X_test)
    test_pred.append(test_labels)
    
    train_proba = cat.predict_proba(X_train)[:,1]
    train_roc.append(roc_auc_score(y_train, train_proba))
    
    train_labels = cat.predict(X_train)
    train_f1.append(f1_score(y_train, train_labels))
    train_acc.append(accuracy_score(y_train, train_labels))
    
    pred_test = cat.predict_proba(X_test)[:,1]
    test_results.append(pred_test)

final_test = np.array(test_results).mean(axis=0)
test_pred = vote(test_pred, weights=[1.,1.,1.])

In [79]:
print(f'train mean accuracy: {np.array(train_acc).mean():.4f}, test accuracy: {accuracy_score(y_test, test_pred):.4f}')
print(f'train mean f1: {np.array(train_f1).mean():.4f}, test accuracy: {f1_score(y_test, test_pred):.4f}')
print(f'train mean auc: {np.array(train_roc).mean():.4f}, test auc: {roc_auc_score(y_test, final_test):.4f}')
# cat model also overfits a bit, but also not critically

In [80]:
cm = confusion_matrix(y_true=y_test, y_pred=test_pred)
conf_df = pd.DataFrame(cm, index=[0,1], columns=[0,1])
plt.figure(figsize=(16,6))
plt.subplot(121)
sns.heatmap(conf_df, cmap="Oranges", annot=True, annot_kws={"size": 16}, fmt='d')
plt.title('Confusion matrix for test')
plt.ylabel('True')
plt.xlabel('Predicted')
plt.show()

In [81]:
roc_plot(y_test, final_test)

In [82]:
# shap importancies
shap_values = cat.get_feature_importance(train_pool, type='ShapValues')

expected_value = shap_values[0,-1]
shap_values = shap_values[:,:-1]

In [83]:
shap.initjs()
shap.force_plot(expected_value, shap_values[0,:], X_fold.iloc[0,:])
# so client with big sum of transactions is more likely to stay, otherhand with low frequency of transations and
# long time inactive he is going to churn

In [89]:
shap.initjs()
shap.summary_plot(shap_values, X_train) # initial, not from kfold loop!
# so Total_Trans_Ct if high -> leads to churn, big sums of transactions leads client to stay, High amount of contancts
# leads client to stay

#### Let's try to retrain models during kfold splits

In [90]:
# experiment
cats = [CatBoostClassifier(cat_features=cat_cols, **params_model)] * 3
test_results, train_roc, test_pred, train_f1, train_acc = [], [], [], [], []
skf = StratifiedKFold(n_splits=3)

i = 0
for train_index, valid_index in skf.split(X_fold, y_fold):
    X_train, X_valid = X_fold.iloc[train_index, :], X_fold.iloc[valid_index, :]
    y_train, y_valid = y_fold.iloc[train_index], y_fold.iloc[valid_index]
    
    if i == 0:
        cats[i].fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
    else: 
        cats[i].fit(X_train, y_train, eval_set=[(X_valid, y_valid)], init_model=cats[i-1])
    test_labels = cats[i].predict(X_test)
    test_pred.append(test_labels)
    
    train_proba = cats[i].predict_proba(X_train)[:,1]
    train_roc.append(roc_auc_score(y_train, train_proba))
    
    train_labels = cats[i].predict(X_train)
    train_f1.append(f1_score(y_train, train_labels))
    train_acc.append(accuracy_score(y_train, train_labels))
    
    pred_test = cats[i].predict_proba(X_test)[:,1]
    test_results.append(pred_test)
    i+=1

final_test = np.array(test_results).mean(axis=0)
test_pred = vote(test_pred, weights=[1.,1.,1.])

In [91]:
print(f'train mean accuracy: {np.array(train_acc).mean():.4f}, test accuracy: {accuracy_score(y_test, test_pred):.4f}')
print(f'train mean f1: {np.array(train_f1).mean():.4f}, test accuracy: {f1_score(y_test, test_pred):.4f}')
print(f'train mean auc: {np.array(train_roc).mean():.4f}, test auc: {roc_auc_score(y_test, final_test):.4f}')
# f1-score a bit better

#### Let's try another approach: weighted sequential ensemble learing - AdaBoost

In [92]:
# again repeat all necessary steps
df = pd.read_csv(data_root) # read
df = df.iloc[:, :-2] # remove redundant columns
df.drop_duplicates(inplace=True) # drop duplicates
df = reduce_memory(df, verbose=0) # reduce memory
df[TARGET_NAME] = df[TARGET_NAME].map({'Existing Customer': 0, 'Attrited Customer': 1}) # map target
df[TARGET_NAME] = df[TARGET_NAME].astype(np.uint8)
df['Gender'] = df['Gender'].map({'M': 0, 'F': 1})
df['Gender'] = df['Gender'].astype(np.uint8)
df.set_index('CLIENTNUM', drop=True, inplace=True) # set client id as index
df = pd.get_dummies(df, prefix=['col1', 'col2', 'col3', 'col4'])
# as Ada Boost algorithm also uses Decision Trees, no data scaling is needed
# split
X, y = df.drop(TARGET_NAME, axis=1), df[TARGET_NAME]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, stratify=y_train, random_state=123)
print(f'Train size: {X_train.shape}, Validation size: {X_valid.shape}, Test size: {X_test.shape}')

In [93]:
model_params = {
                'random_state': 123,  
}

In [96]:
def objective(trial):
    param_trials = {
                    'n_estimators': trial.suggest_int('n_estimators', 50, 800),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 2.),
                    'algorithm': trial.suggest_categorical('algorithm', ['SAMME', 'SAMME.R'])
                    }
    param_trials.update(model_params)
    opt_model = AdaBoostClassifier(**param_trials)
    opt_model.fit(X_train, y_train)
    
    y_pred = opt_model.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    
    return score

In [97]:
optuna.logging.set_verbosity(optuna.logging.FATAL)
study = optuna.create_study(sampler=TPESampler(seed=123), direction="maximize")
study.optimize(objective, n_trials=300, timeout=6000)

print(f'Number of completed trials: {len(study.trials)}')
print('Best trial')
trial = study.best_trial
print(f'Best score: {trial.value}')
print('Best params')
for key, value in trial.params.items():
    print(f'{key}: {value}')

In [98]:
X_fold, X_test, y_fold, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=123)

In [99]:
model_params = {
                'random_state': 123,
                'n_estimators': 351,
                'learning_rate': 1.7243852737318626,
                'algorithm': 'SAMME',
}

In [100]:
# get result probabilities on 3 folds
test_results, train_roc, test_pred, train_f1, train_acc = [], [], [], [], []
skf = StratifiedKFold(n_splits=3)

for train_index, valid_index in skf.split(X_fold, y_fold):
    X_train, X_valid = X_fold.iloc[train_index, :], X_fold.iloc[valid_index, :]
    y_train, y_valid = y_fold.iloc[train_index], y_fold.iloc[valid_index]

    cat = AdaBoostClassifier(**model_params)

    cat.fit(X_train, y_train)
    test_labels = cat.predict(X_test)
    test_pred.append(test_labels)
    
    train_proba = cat.predict_proba(X_train)[:,1]
    train_roc.append(roc_auc_score(y_train, train_proba))
    
    train_labels = cat.predict(X_train)
    train_f1.append(f1_score(y_train, train_labels))
    train_acc.append(accuracy_score(y_train, train_labels))
    
    pred_test = cat.predict_proba(X_test)[:,1]
    test_results.append(pred_test)

final_test = np.array(test_results).mean(axis=0)
test_pred = vote(test_pred, weights=[1.,1.,1.])

In [101]:
print(f'train mean accuracy: {np.array(train_acc).mean():.4f}, test accuracy: {accuracy_score(y_test, test_pred):.4f}')
print(f'train mean f1: {np.array(train_f1).mean():.4f}, test accuracy: {f1_score(y_test, test_pred):.4f}')
print(f'train mean auc: {np.array(train_roc).mean():.4f}, test auc: {roc_auc_score(y_test, final_test):.4f}')
# ada boost is a bit worse